# A. Project Name:  IMDb Successful Movie.
- **Student Name:** Eduardo Galindez.
- **Coding Dojo Bootcamp:** Data Science.
  - **Stack:** Data Enrichment.
- **Date:** September 23th, 2022.

# B. Project Objective
Our Stakeholder Wants More Data!
- After investigating the preview of our data from Part A, the stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).
    - Our stakeholder identified The Movie Database ([TMDB](https://www.themoviedb.org/)) as a great source of financial data . Thankfully, TMDB offers a free API for programmatic access to their data!
- The stakeholder wants we to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

# C. Project Statement


### Specifications:

Our stakeholder would like you to extract and save the results for movies that meet all of the criteria established in Part A of the project.

As a proof-of-concept, they requested we perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

Confirm Your API Function works.

- In order to ensure our function for extracting movie data from TMDB is working, test the function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that the function runs without error and that it returns the correct movie's data for both test ids.

- Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a Exploratory Data Analysis to explore the following questions.

### Exploratory Data Analysis
1. Load in your csv.gz's of results for each year extracted.
 - Concatenate the data into 1 dataframe for the remainder of the analysis.
2. Once you have your data from the API, they would like we to perform some light EDA to show:
 - How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
 - Exclude any movies with 0's for budget AND revenue from the remaining visualizations.
 - How many movies are there in each of the certification categories (G/PG/PG-13/R)?
 - What is the average revenue per certification category?
 - What is the average budget per certification category?

### Deliverable:

After we have joined the tmdb results into 1 dataframe in the EDA Notebook:

- Save a final merged .csv.gz of all of the tmdb api data.
- The file name should be "tmdb_results_combined.csv.gz".
- Make sure this is pushed to our GitHub repository along with all of the code.
- One code file for API calls.
- One code file for EDA.
- Submit the link.

# D. Project Development

## 1.- Libraries & Functions

In [1]:
# Libraries.
import numpy as np
import pandas as pd
import tmdbsimple as tmdb 
import os, time, json
os.makedirs('Data',exist_ok=True)

from tqdm.notebook import tqdm_notebook
#import glob

In [2]:
# Function to get the certification.
def get_movie_certification(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1'] == "US":
            info['certifcation'] = c['certification']     
    return info

In [3]:
# Function to create our .json file.
##  Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/

def write_json(new_data, filename):    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## 2.-  Data & Connection

### 2.1.- API connection


In [4]:
# Loading API credentials.
with open('/Users/eduar/.secret/tmdb_api.json', 'r') as file:
    login = json.load(file)
login.keys()

dict_keys(['api-key'])

In [5]:
# Import credentials.
tmdb.API_KEY =  login['api-key']

In [6]:
# Checking the connection with 'The Avengers'.
the_avengers_movie = tmdb.Movies('tt0848228')
the_avengers_info = the_avengers_movie.info()
the_avengers_info['budget']

220000000

In [7]:
# Checking the connection with 'The Notebook'.
the_notebook_movie = tmdb.Movies('tt0332280')
the_notebook_info = the_notebook_movie.info()
the_notebook_info['budget']

29000000

### 2.2.- Mount and loading data.

In [8]:
# Specify folder for saving data.
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'Chunk data per database',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'Original data',
 'title_akas_combined.csv.gz',
 'title_basics_combined.csv.gz',
 'title_ratings_combined.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [9]:
# Load in the dataframe from Part A:
basics_df = pd.read_csv('./Data/title_basics_combined.csv.gz', low_memory = False)

In [10]:
# Let's drop the column 'endYear' as was reported in Section E of Part A notebook.
basics_df = basics_df.drop(columns=['endYear'])

In [11]:
# Convert 'startYear' column dtype to int.
basics_df['startYear'] = basics_df['startYear'].astype(int)

In [12]:
# Let's confirm we don't have missing data.
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79858 entries, 0 to 79857
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          79858 non-null  object
 1   titleType       79858 non-null  object
 2   primaryTitle    79858 non-null  object
 3   originalTitle   79858 non-null  object
 4   isAdult         79858 non-null  int64 
 5   startYear       79858 non-null  int32 
 6   runtimeMinutes  79858 non-null  int64 
 7   genres          79858 non-null  object
dtypes: int32(1), int64(2), object(5)
memory usage: 4.6+ MB


In [13]:
# General look of the dataframe.
basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0108549,movie,West from North Goes South,West from North Goes South,0,2004,96,"Comedy,Mystery"
1,tt0113026,movie,The Fantasticks,The Fantasticks,0,2000,86,"Musical,Romance"
2,tt0113092,movie,For the Cause,For the Cause,0,2000,100,"Action,Adventure,Drama"
3,tt0114447,movie,The Silent Force,The Silent Force,0,2001,90,Action
4,tt0115937,movie,Consequence,Consequence,0,2000,91,Drama
...,...,...,...,...,...,...,...,...
79853,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
79854,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"
79855,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller
79856,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"


In [14]:
# Create Required Lists for the our function.
YEARS_TO_GET = [2000, 2001]
errors = [ ]

In [15]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Defining the JSON file to store results for year.
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if the file exists.
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it.
    if file_exists == False:
    # Save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as file:
            json.dump([{'imdb_id':0}], file)

    # Saving new year as the current df.        
    df = basics_df.loc[basics_df['startYear'] == YEAR].copy()
    # Saving movie ids to list.
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # Filter out any ids that are already in the JSON_FILE.
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    
    # Start of INNER Loop.
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id.
            temp = get_movie_certification(movie_id)  
            # Append/extend results to existing file using a pre-made function.
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server.
            time.sleep(0.02)
         
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1412 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1529 [00:00<?, ?it/s]

- Total errors: 446


In [16]:
# Let's load data from 2000.
movies_from_2000_df = pd.read_csv('./Data/final_tmdb_data_2000.csv.gz', low_memory = False)
movies_from_2000_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certifcation
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.50,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.10,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.00,0.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.11,1984.0,PG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,tt6174238,0.0,NaN,"{'id': 404302, 'name': 'Cold War Collection', ...",0.0,"[{'id': 80, 'name': 'Crime'}]",NaN,223878.0,cn,冷战,...,0.0,0.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,Cold War,0.0,2.00,2.0,NaN
1204,tt7029820,0.0,NaN,NaN,0.0,[],NaN,604889.0,en,Scream For Christmas,...,0.0,80.0,[],Released,NaN,Scream For Christmas,0.0,0.00,0.0,NaN
1205,tt7197642,0.0,NaN,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,872676.0,en,"Goodbye, Merry-Go-Round",...,0.0,90.0,[],Released,NaN,"Goodbye, Merry-Go-Round",0.0,0.00,0.0,NaN
1206,tt7631368,0.0,/sF0gUHE0YzZNXYugTB2LFxJIppf.jpg,NaN,10000000.0,"[{'id': 27, 'name': 'Horror'}]",NaN,97186.0,fr,"I, Vampire",...,0.0,85.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,"I, Vampire",0.0,6.40,4.0,NR


In [17]:
# Let's load data from 2001.
movies_from_2001_df = pd.read_csv('./Data/final_tmdb_data_2001.csv.gz', low_memory = False)
movies_from_2001_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certifcation
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.0,3.0,NaN
2,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.6,117.0,PG-13
3,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,NaN,1000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.1,28.0,R
4,tt0119004,0.0,/7xrlSPGDO4CDT6IHTctDlkYxTzw.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,37857.0,en,Don's Plum,...,6297.0,89.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Don's Plum,0.0,5.4,63.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,tt7797670,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,956214.0,en,Edmund Kemper Part 2: La Mort C'est La Vie,...,0.0,91.0,[],Released,NaN,Edmund Kemper Part 2: La Mort C'est La Vie,0.0,0.0,0.0,NaN
1285,tt7797790,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,956219.0,en,Edmund Kemper Part 3: La mort sévit,...,0.0,72.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,NaN,Edmund Kemper Part 3: La mort sévit,0.0,0.0,0.0,NaN
1286,tt8665056,0.0,NaN,NaN,0.0,"[{'id': 37, 'name': 'Western'}]",http://skeletoncreekproductions.com/p-movie-br...,885436.0,en,Guns Along The Bravo,...,0.0,85.0,[],Released,Evil came to the Southwest until three blazing...,Guns Along The Bravo,0.0,0.0,0.0,NaN
1287,tt8795764,0.0,NaN,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,0.0,57.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,New Breed,0.0,0.0,0.0,NR


In [18]:
# Let's concatenate both datasets.




### 2.3.- Get Budget, Revenue and Certification per movie tested in Section 2.1.

#### 2.3.1.- The Avengers

In [19]:
# Budget.
budget_avengers_info = the_avengers_info['budget']
print(f'The budget for The Avengers was USD {budget_avengers_info:,}.')

The budget for The Avengers was USD 220,000,000.


In [20]:
# Revenue.
revenue_avengers_info = the_avengers_info['revenue']
print(f'The revenue for The Avengers was USD {revenue_avengers_info:,}.')

The revenue for The Avengers was USD 1,518,815,515.


In [21]:
# Impact of the revenue (Return of Investment).
avengers_performance = round((revenue_avengers_info - budget_avengers_info)/budget_avengers_info *100, 2)
print(f'The ROI for The Avengers was {avengers_performance}%.')

The ROI for The Avengers was 590.37%.


In [22]:
# Certification (MPAA Rating).
avengers_certification = get_movie_certification('tt0848228')
avengers_certification

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 203.07,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

#### 2.3.2.- The Notebook

In [23]:
# Budget.
budget_notebook_info = the_notebook_info['budget']
print(f'The budget for The Notebook was USD {budget_notebook_info:,}.')

The budget for The Notebook was USD 29,000,000.


In [24]:
# Revenue.
revenue_notebook_info = the_notebook_info['revenue']
print(f'The revenue for The Notebook was USD {revenue_notebook_info:,}.')

The revenue for The Notebook was USD 115,603,229.


In [25]:
# Impact of the revenue (Return of Investment).
notebook_performance = round((revenue_notebook_info - budget_notebook_info)/budget_notebook_info *100, 2)
print(f'The ROI for The Notebook was {notebook_performance}%.')

The ROI for The Notebook was 298.63%.


In [26]:
# Certification (MPAA Rating).
notebook_certification = get_movie_certification('tt0332280')
notebook_certification

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 56.321,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [27]:
# Summary between both movies.
test_ids = ['tt0848228', 'tt0332280']
results = []

for movie_id in test_ids:  
    try:
        movie_info = get_movie_certification(movie_id)
        results.append(movie_info)        
    except: 
        pass
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certifcation
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.705,27524,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,9820,PG-13


## 3.- Visual Data Exploration
- Xxxxx

# E. Conclusions

- Xxxxx